In [ ]:
import random
import requests
import re
import logging
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pytz
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder


try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
except:
    nlp = None

logging.basicConfig(level=logging.INFO)

class VastChatbot:
    def __init__(self):
        self.training_data = {
            "greeting": [
                "hi", "hello", "hey", "good morning", "good evening",
                "yo", "hiya", "sup", "what's up", "heya", "hi there", "hello there"
            ],
            "goodbye": ["bye", "goodbye", "see you", "farewell"],
            "thanks": ["thanks", "thank you", "appreciate it"],
            "weather": [
                "what's the weather", "weather forecast", "is it raining",
                "temperature outside", "weather in", "how's the weather in London"
            ],
            "joke": ["tell me a joke", "make me laugh", "joke please"],
            "time": [
                "what time is it", "current time", "time now",
                "time in", "what time is it in Tokyo"
            ],
            "help": ["help", "commands", "how to use"],
            "about": [
                "how can you help me", "who are you", "what can you do",
                "tell me about yourself", "your features"
            ],
            "unknown": ["blargh", "random words", "nothing to say"]
        }

        self.vectorizer = TfidfVectorizer()
        self.model = LogisticRegression()
        sentences, labels = [], []
        for intent, examples in self.training_data.items():
            sentences.extend(examples)
            labels.extend([intent] * len(examples))
        X = self.vectorizer.fit_transform(sentences)
        self.model.fit(X, labels)

        self.weather_api_key = "YOUR WEATHER API KEY f"
        self.user_memory = {"name": None, "city": None}

    def predict_intent(self, user_input):
        X_test = self.vectorizer.transform([user_input])
        probs = self.model.predict_proba(X_test)[0]
        max_prob = max(probs)
        intent = self.model.classes_[probs.argmax()]
        logging.info(f"Intent prediction: {intent} ({max_prob:.2f})")
        return intent if max_prob > 0.2 else "unknown"

    def extract_city(self, text):
        if nlp:
            doc = nlp(text)
            for ent in doc.ents:
                if ent.label_ == "GPE":
                    return ent.text

        patterns = [
            r"weather in ([a-zA-Z\s]+)",
            r"time in ([a-zA-Z\s]+)",
            r"current time in ([a-zA-Z\s]+)",
            r"in ([a-zA-Z\s]+) time"
        ]
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                return match.group(1).strip()
        return None

    def get_response(self, intent, user_input):
        # Memory input recognition
        name_match = re.search(r"my name is ([a-zA-Z\s]+)", user_input.lower())
        if name_match:
            self.user_memory["name"] = name_match.group(1).title().strip()
            return f"Nice to meet you, {self.user_memory['name']}!"

        city_match = re.search(r"i live in ([a-zA-Z\s]+)", user_input.lower())
        if city_match:
            self.user_memory["city"] = city_match.group(1).title().strip()
            return f"Got it! I'll remember that you're from {self.user_memory['city']}."

        if intent == "greeting":
            name = self.user_memory.get("name")
            return f"Hello {name}!" if name else random.choice(["Hello!", "Hi there!", "Hey!"])
        elif intent == "goodbye":
            return random.choice(["Goodbye!", "See you later!", "Take care!"])
        elif intent == "thanks":
            return random.choice(["You're welcome!", "No problem!"])
        elif intent == "joke":
            return self.fetch_joke()
        elif intent == "weather":
            city = self.extract_city(user_input.lower()) or self.user_memory.get("city") or "Jalna"
            return self.fetch_weather(city)
        elif intent == "time":
            city = self.extract_city(user_input.lower()) or self.user_memory.get("city")
            return self.get_time(city)
        elif intent == "help":
            return (
                "I can help you with:\n"
                "- Weather info (e.g., 'weather in Paris')\n"
                "- Jokes\n"
                "- Current time (e.g., 'time in Tokyo')\n"
                "- I can also remember your name or city!"
            )
        elif intent == "about":
            return (
                "I'm VastChatbot! I can tell jokes, report weather, give time by city, "
                "remember your name/city, and assist with general questions. Try asking me anything!"
            )
        else:
            return "Sorry, I didn’t understand that."

    def get_time(self, city):
        try:
            if city:
                city = city.lower().strip()
                city_timezones = {
                    "mumbai": "Asia/Kolkata",
                    "delhi": "Asia/Kolkata",
                    "kolkata": "Asia/Kolkata",
                    "pune": "Asia/Kolkata",
                    "tokyo": "Asia/Tokyo",
                    "new york": "America/New_York",
                    "london": "Europe/London",
                    "paris": "Europe/Paris",
                    "berlin": "Europe/Berlin",
                    "los angeles": "America/Los_Angeles",
                    "sydney": "Australia/Sydney",
                    "dubai": "Asia/Dubai",
                    "singapore": "Asia/Singapore",
                    "chicago": "America/Chicago",
                    "toronto": "America/Toronto",
                    "moscow": "Europe/Moscow"
                }

                timezone_str = city_timezones.get(city)
                if timezone_str:
                    now = datetime.now(pytz.timezone(timezone_str)).strftime("%H:%M:%S")
                    return f"The current time in {city.title()} is {now} ({timezone_str})."
                else:
                    #  Use timezonefinder 
                    geolocator = Nominatim(user_agent="vast_chatbot")
                    location = geolocator.geocode(city)
                    if location:
                        tf = TimezoneFinder()
                        tz_name = tf.timezone_at(lat=location.latitude, lng=location.longitude)
                        if tz_name:
                            now = datetime.now(pytz.timezone(tz_name)).strftime("%H:%M:%S")
                            return f"The current time in {city.title()} is {now} ({tz_name})."
                    return f"Sorry, I don’t know the timezone for {city.title()}."
            else:
                now = datetime.utcnow().strftime("%H:%M:%S")
                return f"The current time is {now} UTC."
        except Exception as e:
            logging.error(f"Time lookup failed: {e}")
            return "Sorry, I couldn't get the time for that city."

    def fetch_joke(self):
        try:
            response = requests.get("https://official-joke-api.appspot.com/random_joke")
            data = response.json()
            return f"{data['setup']} ... {data['punchline']}"
        except Exception:
            return "Why don’t skeletons fight each other? They don’t have the guts."

    def fetch_weather(self, city):
        if not self.weather_api_key:
            return "Weather API key not set."
        try:
            url = (
                f"http://api.openweathermap.org/data/2.5/weather?q={city}"
                f"&appid={self.weather_api_key}&units=metric"
            )
            response = requests.get(url)
            data = response.json()
            if data.get("cod") != 200:
                return f"Couldn't find weather for '{city}'."
            temp = data['main']['temp']
            desc = data['weather'][0]['description']
            return f"The weather in {city.title()} is {desc} with {temp}°C."
        except Exception:
            return "Failed to fetch weather data."

if __name__ == "__main__":
    bot = VastChatbot()
    print("Hello Pravin how can i help you today (type 'exit' to quit)\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Bot: Goodbye!")
            break
        intent = bot.predict_intent(user_input)
        response = bot.get_response(intent, user_input)
        print("Bot:", response)


Hello Pravin how can i help you today (type 'exit' to quit)



You:  hi


INFO:root:Intent prediction: greeting (0.50)


Bot: Hello!


You:  how can you help me


...

INFO:root:Intent prediction: weather (0.41)


Bot: Couldn't find weather for 'chhatrapati sambhajinagar'.


You:  weather in aurangabad


INFO:root:Intent prediction: weather (0.41)


Bot: The weather in Aurangabad is overcast clouds with 26.89°C.
